# Notebook version of 2.3_dps.py, which runs over a single tile
#### cant get 2.3_dps.py to work, so alternate approach is to loop over tile ids and return indiv tile CSVs of ATL08

In [6]:
#import pdal
import json
import os
import glob
import time

import itertools

import pandas as pd
from pyproj import CRS, Transformer

# From CovariateUtils
import numpy
from rasterio import enums
from rasterio.io import MemoryFile
from rasterio.crs import CRS
from rasterio.vrt import WarpedVRT
from rasterio.warp import array_bounds, calculate_default_transform
import geopandas

import folium
import shapely as shp

import argparse

from maap.maap import MAAP
maap = MAAP()

import sys
sys.path.append('/projects/code/icesat2_boreal/notebooks/3.Gridded_product_development')
if False:
    
    sys.path.append('/projects/code/icesat2_boreal/notebooks/2.ICESat-2_processing')

    #TODO: how to get this import right if its in a different dir
    from CovariateUtils import get_index_tile

import ExtractUtils
import FilterUtils



In [18]:
def get_atl08_csv_list(dps_dir_csv, seg_str, csv_list_fn):
    print(dps_dir_csv + "/**/ATL08*" + seg_str + ".csv") 
    #seg_str="_30m"
    print('Running glob.glob to return a list of csv paths...')
    all_atl08_csvs = glob.glob(dps_dir_csv + "/**/ATL08*" + seg_str + ".csv", recursive=True)
    print(len(all_atl08_csvs))
    all_atl08_csvs_df = pd.DataFrame({"path": all_atl08_csvs})
    all_atl08_csvs_df.to_csv(csv_list_fn)
    return(all_atl08_csvs_df)
    
def get_topo_stack_list(dps_dir_topo, topo_stack_list_fn):
    all_topo_stacks = glob.glob(dps_dir_topo + "/**/Copernicus_*_covars_cog_topo_stack.tif", recursive=True)
    print(len(all_topo_stacks))
    all_topo_stacks_df = pd.DataFrame({"path": all_topo_stacks})
    all_topo_stacks_df.to_csv(topo_stack_list_fn)
    return(all_topo_stacks_df)

def get_topo_stack_fn(topo_stack_list_fn, in_tile_num):
    # Find most recent topo stack path for tile in list of topo stack paths 
    all_topo_stacks_df = pd.read_csv(topo_stack_list_fn)
    stack_for_tile = all_topo_stacks_df[all_topo_stacks_df['path'].str.contains("Copernicus_"+str(in_tile_num))]
    [print(i) for i in stack_for_tile.path.to_list()]
    topo_stack_fn = stack_for_tile.path.to_list()[0]
    return(topo_stack_fn)

# Set paths and make input tile list

In [9]:
maap_query = True

dps_dir_topo =       '/projects/my-private-bucket/dps_output/do_topo_stack_3-1-5_ubuntu/ops/2021/07'#'/projects/r2d2/dps_output/do_topo_stack_3-1-5_ubuntu/master/2021/06/18'
#dps_dir_csv =        '/projects/r2d2/dps_output/run_extract_atl08_orig_ubuntu/master/2021/06/17' #'/projects/jabba/dps_output/2.3_output'#
#dps_dir_csv =        '/projects/my-private-bucket/dps_output/do_topo_stack_3-1-5_ubuntu/ops/2021/07/15'
dps_dir_csv =        '/projects/shared-buckets/montesano/run_extract_atl08_orig_ubuntu/master/2021'

output_dir =         '/projects/my-public-bucket/out_tiles'
topo_stack_list_fn = '/projects/my-public-bucket/topo_stack_list.csv'
#csv_list_fn =        '/projects/jabba/data/extract_atl08_csv_list.csv'
csv_list_fn =        '/projects/my-public-bucket/extract_atl08_csv_list.csv'

ALL_BOREAL = False
TEST = False
do_30m = True
extract_covars = True

in_tile_fn =         '/projects/my-public-bucket/boreal_grid_albers90k_gpkg.gpkg' #'/projects/maap-users/alexdevseed/boreal_tiles.gpkg' # '/projects/jabba/data/grid_boreal_albers_100k.gpkg'# 
in_tile_layer =       'grid_boreal_albers90k_gpkg' #'boreal_tiles_albers'

thresh_h_can = 100
thresh_h_dif = 100
month_min = 6
month_max = 9

date_start = '06-01'
date_end = '09-30'
years_list = [2018, 2019, 2020, 2021]

boreal_tile_index = geopandas.read_file(in_tile_fn)

if ALL_BOREAL:
    region_str = 'All Boreal'
    # All Boreal tiles
    INPUT_TIL_NUM_LIST_FULL = boreal_tile_index['layer'].astype(int).tolist()
    print(len(INPUT_TIL_NUM_LIST_FULL))
else:
    region_str = 'NA Boreal subset'
    # NA boreal tiles subset
    boreal_tile_index_subset = boreal_tile_index.to_crs(4326).cx[-170:-50, 50:75]
    INPUT_TILE_NUM_LIST = boreal_tile_index_subset['layer'].astype(int).tolist()

#INPUT_TILE_NUM_LIST = INPUT_TILE_NUM_LIST[100:151]

print(f'Length of list for {region_str}:\n', len(INPUT_TILE_NUM_LIST))
print('\n',INPUT_TILE_NUM_LIST)


Length of list for NA Boreal subset:
 1305

 [358, 389, 420, 421, 451, 452, 453, 454, 455, 456, 490, 491, 492, 493, 494, 495, 496, 529, 530, 531, 532, 533, 534, 535, 536, 569, 570, 571, 572, 573, 574, 575, 576, 577, 610, 611, 612, 613, 614, 615, 616, 617, 618, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 695, 696, 697, 698, 699, 700, 701, 702, 703, 704, 705, 706, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 785, 786, 787, 788, 789, 790, 791, 792, 793, 794, 795, 796, 797, 830, 831, 832, 833, 834, 835, 836, 837, 838, 839, 840, 841, 842, 843, 844, 876, 877, 878, 879, 880, 881, 882, 883, 884, 885, 886, 887, 888, 889, 890, 924, 925, 926, 927, 928, 929, 930, 931, 932, 933, 934, 935, 936, 937, 938, 975, 976, 977, 978, 979, 980, 981, 982, 983, 984, 985, 986, 987, 988, 1030, 1031, 1032, 1033, 1034, 1035, 1036, 1037, 1038, 1039, 1040, 1041, 1042, 1043, 1044, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1096, 1097, 1098, 1099, 1141, 1142, 1143, 1144, 1145, 

#### Build the list of topo tiles (must do after every DPS run)

In [5]:
all_topo_stacks_df = get_topo_stack_list(dps_dir_topo, topo_stack_list_fn)
all_topo_stacks_df.head()

1256


,path
0,/projects/my-private-bucket/dps_output/do_topo...
1,/projects/my-private-bucket/dps_output/do_topo...
2,/projects/my-private-bucket/dps_output/do_topo...
3,/projects/my-private-bucket/dps_output/do_topo...
4,/projects/my-private-bucket/dps_output/do_topo...


# Loop over input tiles to output ATL08 tile CSVs

#### In here, when run for the first time, you build the list of extracted CSV paths from DPS and save all paths in a list - glob.glob takes a while

In [20]:
%%time
#INPUT_TILE_NUM_LIST = range(30823,30824) #, 30543, 30821, 30822, 30823]:
#INPUT_TILE_NUM_LIST = [14151]#[30542]#[30543]
#INPUT_TILE_NUM_LIST = INPUT_TILE_NUM_LIST[100:151]
INPUT_TILE_NUM_LIST = [456]
for in_tile_num in INPUT_TILE_NUM_LIST:
    
    # TODO: make this an arg
    years_list = [2018, 2019, 2020, 2021]
    
    seg_str = '_100m'
    if do_30m:
        seg_str = '_30m'
    if TEST:
        seg_str = '' 
    
    if maap_query and dps_dir_csv is not None:
        
        print("\nDoing MAAP query by tile bounds to find all intersecting ATL08 ")
        # Get a list of all ATL08 H5 granule names intersecting the tile (this will be a small list)
        all_atl08_for_tile = ExtractUtils.maap_search_get_h5_list(tile_num=in_tile_num, tile_fn=in_tile_fn, layer=in_tile_layer, DATE_START=date_start, DATE_END=date_end, YEARS=years_list)
        print([os.path.basename(f) for f in all_atl08_for_tile])      
        
        if not os.path.isfile(csv_list_fn):
            all_atl08_csvs_df = get_atl08_csv_list(dps_dir_csv, seg_str, csv_list_fn)
        else:
            print(f"\tReading existing list of ATL08 CSVs: {csv_list_fn}")
            all_atl08_csvs_df = pd.read_csv(csv_list_fn)
            
        # Find the ATL08 CSVs from extract that are associated with the ATL08 granules that intersect this tile
        # These CSvs are nested deep after DPS runs
        # They should match all the ATL08 granules, but probably wont bc: (1) did DPS for ATL08 30m fully complete with no fails? (2) did DPS for extract fully complete with no fails?
        all_atl08_csvs_FOUND, all_atl08_csvs_NOT_FOUND = FilterUtils.find_atl08_csv_tile(all_atl08_for_tile, all_atl08_csvs_df, seg_str) 
        
        #all_atl08_csvs_FOUND = [x for x in all_atl08_h5_with_csvs_for_tile if x not in all_atl08_csvs_NOT_FOUND]
        print("\t# of ATL08 CSV found for tile {}: {}".format(in_tile_num, len(all_atl08_csvs_FOUND)))
        print("\t# of ATL08 CSV NOT found for tile {}: {}".format(in_tile_num, len(all_atl08_csvs_NOT_FOUND)))
        if len(all_atl08_csvs_FOUND) == 0:
            print('\tNo ATL08 extracted for this tile.')
            continue
        
        # Merge all ATL08 CSV files for the current tile into a pandas df
        print("Creating pandas data frame...")
        atl08 = pd.concat([pd.read_csv(f) for f in all_atl08_csvs_FOUND ], sort=False, ignore_index=True)
        atl08 = FilterUtils.prep_filter_atl08_qual(atl08)

        print("\nFiltering by tile: {}".format(in_tile_num))
        # Get tile bounds as xmin,xmax,ymin,ymax
        tile = ExtractUtils.get_index_tile(in_tile_fn, in_tile_num, buffer=0, layer=in_tile_layer)
        in_bounds = FilterUtils.reorder_4326_bounds(tile)
        print(in_bounds)
        
        # Now filter ATL08 obs by tile bounds (changed to a geopandas clip instead of simple bounds approach)
        
        atl08 = FilterUtils.filter_atl08_bounds_clip(atl08, tile['geom_4326'])
        #atl08 = FilterUtils.filter_atl08_bounds(atl08_df=atl08, in_bounds=in_bounds)

    elif maap_query and dps_dir_csv is None:
        print("\nNo DPS dir specified: cant get ATL08 CSV list to match with tile bound results from MAAP query.\n")
        #os._exit(1)
        continue
    else:
        # Filter by bounds: EPT with a the bounds from an input tile
        atl08 = FilterUtils.filter_atl08_bounds_tile_ept(in_ept_fn, in_tile_fn, in_tile_num, in_tile_layer, output_dir, return_pdf=True)
    
    # Filter by quality
    atl08_pdf_filt = FilterUtils.filter_atl08_qual(atl08, SUBSET_COLS=True, DO_PREP=False,
                                                       subset_cols_list=['rh25','rh50','rh60','rh70','rh75','rh80','rh90','h_can','h_max_can'], 
                                                       filt_cols=['h_can','h_dif_ref','m','msw_flg','beam_type','seg_snow', 'seg_landcov'], 
                                                       thresh_h_can=100, thresh_h_dif=100, month_min=6, month_max=9)
    atl08=None
    
    # Convert to geopandas data frame in lat/lon
    atl08_gdf = geopandas.GeoDataFrame(atl08_pdf_filt, geometry=geopandas.points_from_xy(atl08_pdf_filt.lon, atl08_pdf_filt.lat), crs='epsg:4326')
    out_name_stem = "atl08_filt"
    atl08_pdf_filt=None
    
    if extract_covars and len(atl08_gdf) > 0:
        ### Below here should be re-worked to follow final chunk of nb 2.3 (6/15/2021)
        #
        # Extract topo covar values to ATL08 obs (doing a reproject to tile crs)
        # TODO: consider just running 3.1.5_dpy.py here to produce this topo stack right before extracting its values
        #topo_covar_fn = do_3_1_5_dp.main(in_tile_fn=in_tile_fn, in_tile_num=in_tile_num, tile_buffer_m=120, in_tile_layer=in_tile_layer, topo_tile_fn='https://maap-ops-dataset.s3.amazonaws.com/maap-users/alexdevseed/dem30m_tiles.geojson')
        topo_covar_fn = FilterUtils.get_topo_stack_fn(topo_stack_list_fn, in_tile_num)
       
        if topo_covar_fn is not None:
            print("\nExtract topo covars...")
            print(topo_covar_fn)
            atl08_gdf = ExtractUtils.extract_value_gdf(topo_covar_fn, atl08_gdf, ["elevation","slope","tsri","tpi", "slopemask"], reproject=True)
            out_name_stem = out_name_stem + "_topo"

        if False:
            # Extract landsat covar values to ATL08 obs
            # TODO: consider just running 3.1.2_dpy.py here
            landsat_covar_fn = do_3_1_2_dps.main(in_tile_fn=in_tile_fn, in_tile_num=in_tile_num, in_tile_layer=in_tile_layer, sat_api='https://landsatlook.usgs.gov/sat-api', local=args.local)
            atl08_gdf = ExtractUtils.extract_value_gdf(landsat_covar_fn, atl08_gdf, ['Blue', 'Green', 'Red', 'NIR', 'SWIR', 'NDVI', 'SAVI', 'MSAVI', 'NDMI', 'EVI', 'NBR', 'NBR2', 'TCB', 'TCG', 'TCW', 'ValidMask', 'Xgeo', 'Ygeo'], reproject=False)
            out_name_stem = out_name_stem + "_landsat"
        
    if len(atl08_gdf) == 0:
        print(f"No ATL08 obs. for tile {in_tile_num}")
    else:
        # CSV the file
        cur_time = time.strftime("%Y%m%d") #"%Y%m%d%H%M%S"
        out_fn = os.path.join(output_dir, out_name_stem + "_" + cur_time + "_" + str(in_tile_num) )
        atl08_gdf.to_csv(out_fn+".csv", index=False, encoding="utf-8-sig")
        atl08_gdf.to_file(out_fn+'.geojson', driver="GeoJSON")

        print("Wrote output csv/geojson of filtered ATL08 obs with topo and Landsat covariates for tile {}: {}".format(in_tile_num, out_fn) )


Doing MAAP query by tile bounds to find all intersecting ATL08 
	TILE_NUM: 456 (-55.6319579668971,49.2005565823239,-54.078910165254506,50.2091605991493)
		# ATL08 for tile 456: 39
['ATL08_20190607074217_10700302_003_01.h5', 'ATL08_20190611073358_11310302_003_01.h5', 'ATL08_20190617193308_12300306_003_01.h5', 'ATL08_20190621192447_12910306_003_01.h5', 'ATL08_20190710060947_01860402_003_01.h5', 'ATL08_20190716180900_02850406_003_01.h5', 'ATL08_20190720180042_03460406_003_01.h5', 'ATL08_20190724175225_04070406_003_01.h5', 'ATL08_20190804045421_05670402_003_01.h5', 'ATL08_20190808044601_06280402_003_01.h5', 'ATL08_20190812043741_06890402_003_01.h5', 'ATL08_20190814164511_07270406_003_01.h5', 'ATL08_20190818163653_07880406_003_01.h5', 'ATL08_20190822162835_08490406_003_01.h5', 'ATL08_20190902033027_10090402_003_01.h5', 'ATL08_20190906032206_10700402_003_01.h5', 'ATL08_20190910031347_11310402_003_01.h5', 'ATL08_20190916151258_12300406_003_01.h5', 'ATL08_20190920150439_12910406_003_01.h5', '

# Map checks

In [21]:
atl08_gdf.info()
boreal_tile_index_orig = geopandas.read_file(in_tile_fn)

#tile_parts = ExtractUtils.get_index_tile(in_tile_fn, in_tile_num, buffer=0, layer=in_tile_layer)
#atl08_filt_test = pd.read_csv('/projects/jabba/data/atl08_filt_30543_topo_landsat_20210608185220.csv')
#atl08_filt_test_gdf = geopandas.GeoDataFrame(atl08_filt_test, crs='epsg:4326', geometry=geopandas.points_from_xy(atl08_filt_test.lon, atl08_filt_test.lat))

# Reformat bbox
bbox_list = tile["bbox_4326"]
print(bbox_list)
center = tile["geom_orig"].centroid.to_crs(4326)

m = folium.Map(
    # Zoom to center
    location=[center.y,center.x],
    # Zoom to corner
    #location=[bbox_list[1], bbox_list[2]],
    tiles="cartodbpositron",
    zoom_start=8,
)

geom_style = {'fillColor': 'green', 'color': 'green'}
bbox_style = {'fillColor': '#ff0000', 'color': '#ff0000'}
bbox_buf_style = {'fillColor': '#fdae61', 'color': '#fdae61'}

#folium.GeoJson(boreal_tile_index_orig, name='tiles').add_to(m)

folium.GeoJson(atl08_gdf, name="ATL08").add_to(m)
#folium.GeoJson(atl08_filt_test_gdf, name="ATL08").add_to(m)
folium.GeoJson(shp.geometry.box(*bbox_list), 
               name="bbox",
               style_function=lambda x:bbox_style).add_to(m)
folium.GeoJson(tile['geom_4326'], name='tile', style_function=lambda x:geom_style).add_to(m)
m

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 135 entries, 18174 to 233128
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   lon        135 non-null    float64 
 1   lat        135 non-null    float64 
 2   rh25       135 non-null    float64 
 3   rh50       135 non-null    float64 
 4   rh60       135 non-null    float64 
 5   rh70       135 non-null    float64 
 6   rh75       135 non-null    float64 
 7   rh80       135 non-null    float64 
 8   rh90       135 non-null    float64 
 9   h_can      135 non-null    float64 
 10  h_max_can  135 non-null    float64 
 11  geometry   135 non-null    geometry
 12  elevation  135 non-null    float64 
 13  slope      135 non-null    float64 
 14  tsri       135 non-null    float64 
 15  tpi        135 non-null    float64 
 16  slopemask  135 non-null    float64 
dtypes: float64(16), geometry(1)
memory usage: 19.0 KB
[-55.6319579668971, 49.2005565823239, -54.0789

# Debugging...

In [62]:
from pprint import pprint
pprint(tile)
print(in_tile_num)

{'bbox_4326': [-55.6319579668971,
               49.2005565823239,
               -54.078910165254506,
               50.2091605991493],
 'bbox_4326_buffered': [-55.6319579668971,
                        49.2005565823239,
                        -54.078910165254506,
                        50.2091605991493],
 'bbox_orig': [4598522.0, 7263304.0, 4688522.0, 7353304.0],
 'geom_4326':      layer                                           geometry
455  456.0  POLYGON ((-54.07891 49.97415, -54.44753 49.200...,
 'geom_4326_buffered': 455    POLYGON ((-54.07891 49.97415, -54.44753 49.200...
dtype: geometry,
 'geom_orig':      layer                                           geometry
455  456.0  POLYGON ((4598522.000 7353304.000, 4688522.000...,
 'geom_orig_buffered': 455    POLYGON ((4598522.000 7353304.000, 4688522.000...
dtype: geometry,
 'tile_crs': CRS.from_wkt('PROJCS["unknown",GEOGCS["unknown",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101]],PRIMEM["Greenwich",

In [7]:
if False:
    import pandas as pd
    import FilterUtils

    all_atl08_csvs_FOUND = ['/projects/r2d2/dps_output/run_extract_atl08_orig_ubuntu/master/2021/06/17/22/56/24/607120/ATL08_30m_20190621110851_12860302_003_01_30m.csv', 
                            '/projects/r2d2/dps_output/run_extract_atl08_orig_ubuntu/master/2021/06/17/23/12/32/945664/ATL08_30m_20190818082057_07830402_003_01_30m.csv', 
                            '/projects/r2d2/dps_output/run_extract_atl08_orig_ubuntu/master/2021/06/17/23/11/17/197105/ATL08_30m_20190822081238_08440402_003_01_30m.csv']
    atl08_tmp = pd.concat([pd.read_csv(f) for f in all_atl08_csvs_FOUND ], sort=False, ignore_index=True)
    in_bounds = [-117.10749852280769, -116.50936927974429, 50.78795362739066, 51.16389512140189]
    atl08_tmp = atl08_tmp[(atl08_tmp.lat > in_bounds[2]) &
                           (atl08_tmp.lat < in_bounds[3]) &
                           (atl08_tmp.lon > in_bounds[0]) &
                           (atl08_tmp.lon < in_bounds[1])
                                   ]
    
    print('DEBUG FilterUtils.prep_filter_atl08_qual')
    atl08_tmp = FilterUtils.prep_filter_atl08_qual(atl08_tmp)
    print(len(atl08_tmp))
  

In [63]:
#### Check the CSVs produced from extract_atl08 for a given tile id
all_atl08_csvs_df = None
if os.path.isfile(csv_list_fn):

    all_atl08_for_tile = ExtractUtils.maap_search_get_h5_list(tile_num=in_tile_num, tile_fn=in_tile_fn, layer=in_tile_layer, DATE_START=date_start, DATE_END=date_end, YEARS=years_list)
    all_atl08_csvs_df = pd.read_csv(csv_list_fn)

    print([print(i) for i in all_atl08_for_tile])

    for g in all_atl08_for_tile:
        s = g.split("_")[1]
        f = all_atl08_csvs_df[all_atl08_csvs_df['path'].str.contains(s)]
        if len(f) > 0:
            print(f.path.iloc[0])
else:
    print(f'Need to create the csv list file: {csv_list_fn}')
    
# Check for duplicate CSV
if all_atl08_csvs_df is not None:
    print(f'Length of CSV list from extract_atl08: {len(all_atl08_csvs_df)}')
    just_basenames = [os.path.basename(all_atl08_csvs_df.path[i]) for i in range(0,len(all_atl08_csvs_df))]
    no_dups = set(just_basenames)

    print(f'Length of CSV list from extract_atl08 after removing duplicates: {len(no_dups)}')

    # Check, and get list of duplicated ATL08 CSVs
    import collections
    dups = [item for item, count in collections.Counter(just_basenames).items() if count > 1]
    print(f'# of duplicates in CSV: {len(dups)}')
    len(dups)
    dups[0:10]

	TILE_NUM: 456 (-55.6319579668971,49.2005565823239,-54.078910165254506,50.2091605991493)
		# ATL08 for tile 456: 39
ATL08_20190607074217_10700302_003_01.h5
ATL08_20190611073358_11310302_003_01.h5
ATL08_20190617193308_12300306_003_01.h5
ATL08_20190621192447_12910306_003_01.h5
ATL08_20190710060947_01860402_003_01.h5
ATL08_20190716180900_02850406_003_01.h5
ATL08_20190720180042_03460406_003_01.h5
ATL08_20190724175225_04070406_003_01.h5
ATL08_20190804045421_05670402_003_01.h5
ATL08_20190808044601_06280402_003_01.h5
ATL08_20190812043741_06890402_003_01.h5
ATL08_20190814164511_07270406_003_01.h5
ATL08_20190818163653_07880406_003_01.h5
ATL08_20190822162835_08490406_003_01.h5
ATL08_20190902033027_10090402_003_01.h5
ATL08_20190906032206_10700402_003_01.h5
ATL08_20190910031347_11310402_003_01.h5
ATL08_20190916151258_12300406_003_01.h5
ATL08_20190920150439_12910406_003_01.h5
ATL08_20200604142130_10700702_003_01.h5
ATL08_20200608141310_11310702_003_01.h5
ATL08_20200615021219_12300706_003_01.h5
ATL0